<a href="https://colab.research.google.com/github/Amira126/Hugging-Face-Platform-and-its-API/blob/main/Text_to_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Exploring Text to Speech Models**
- suno/bark-small
- microsoft/speecht5_tts

In [1]:
!pip install transformers

In [2]:
from transformers import pipeline

In [29]:
text = "Hi, I am Amira, a Data Scientist who looks for a Data Science Job where I can contribute to and learn from."

pipe = pipeline('text-to-speech', model = "suno/bark-small", device="cuda")

output = pipe(text)

/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [6]:
output

{'audio': array([[6.37232675e-04, 6.65912958e-05, 2.33184459e-04, ...,
         1.12047065e-02, 1.18566090e-02, 1.20294392e-02]], dtype=float32),
 'sampling_rate': 24000}

In [31]:
from IPython.display import Audio

Audio(output["audio"], rate=output["sampling_rate"])

--------------------------------------------------------------------------

## **Microsoft SpeechT5ForTextToSpeech**

### Loading speaker embeddings (512-element) X-vectors to choose voice and accent.

The CMU ARCTIC dataset divides the utterances among the following speakers:

- bdl (US male)
- slt (US female)
- jmk (Canadian male)
- awb (Scottish male)
- rms (US male)
- clb (US female)
- ksp (Indian male)


In [34]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [35]:
from datasets import load_dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

README.md:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

cmu-arctic-xvectors.py:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/21.3M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/7931 [00:00<?, ? examples/s]

In [37]:
embeddings_dataset

Dataset({
    features: ['filename', 'xvector'],
    num_rows: 7931
})

In [51]:
#lookup dictionary for embedding access using filename
lookup_dict = {row['filename']: row['xvector'] for row in embeddings_dataset}
xvector = lookup_dict.get('cmu_us_slt_arctic-wav-arctic_b0251')

In [52]:
xvector

[-0.08077696710824966,
 -0.025941435247659683,
 0.03947367146611214,
 0.03310742229223251,
 0.033730022609233856,
 -0.0314822643995285,
 -0.0413568876683712,
 -0.05451672896742821,
 0.024007407948374748,
 0.02035859413444996,
 -0.05969646945595741,
 -0.0694633200764656,
 0.06325722485780716,
 0.041544109582901,
 0.026377927511930466,
 0.05767466872930527,
 -0.010624540969729424,
 0.01138684805482626,
 0.011506225913763046,
 0.002629663795232773,
 0.038459789007902145,
 0.032875724136829376,
 -0.014962167479097843,
 -0.03676370158791542,
 -0.06186627224087715,
 -0.008634094148874283,
 -0.04921814799308777,
 0.0003026462800335139,
 0.06622450798749924,
 0.011566586792469025,
 -0.0025781532749533653,
 0.0380621999502182,
 0.04225722700357437,
 -0.02358473464846611,
 0.015233969315886497,
 -0.07529228180646896,
 0.04357556626200676,
 0.06701329350471497,
 -0.07443073391914368,
 -0.057002272456884384,
 0.024953775107860565,
 -0.051099635660648346,
 0.01766943372786045,
 0.01874811388552189,

In [53]:
speaker_embeddings = xvector
speaker_embeddings = torch.tensor(speaker_embeddings).unsqueeze(0)

In [54]:
speaker_embeddings

tensor([[-8.0777e-02, -2.5941e-02,  3.9474e-02,  3.3107e-02,  3.3730e-02,
         -3.1482e-02, -4.1357e-02, -5.4517e-02,  2.4007e-02,  2.0359e-02,
         -5.9696e-02, -6.9463e-02,  6.3257e-02,  4.1544e-02,  2.6378e-02,
          5.7675e-02, -1.0625e-02,  1.1387e-02,  1.1506e-02,  2.6297e-03,
          3.8460e-02,  3.2876e-02, -1.4962e-02, -3.6764e-02, -6.1866e-02,
         -8.6341e-03, -4.9218e-02,  3.0265e-04,  6.6225e-02,  1.1567e-02,
         -2.5782e-03,  3.8062e-02,  4.2257e-02, -2.3585e-02,  1.5234e-02,
         -7.5292e-02,  4.3576e-02,  6.7013e-02, -7.4431e-02, -5.7002e-02,
          2.4954e-02, -5.1100e-02,  1.7669e-02,  1.8748e-02,  3.5287e-02,
         -1.1703e-01, -1.8289e-02,  1.2517e-02, -7.3136e-02,  5.5626e-02,
          2.1417e-02,  9.6340e-03,  8.5882e-03,  4.0276e-02, -9.6015e-02,
          1.8325e-02,  1.0046e-02,  3.5027e-02,  7.4576e-03, -4.0970e-03,
          1.9041e-02, -1.0236e-02, -1.5458e-02,  4.2694e-02,  7.6848e-03,
          2.7353e-02,  1.4632e-02, -3.

In [59]:
text = "Hi, I am Amira, a Data Scientist, a lifelong learner."

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan, set_seed
from IPython.display import Audio
import torch

In [ ]:
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [ ]:
inputs = processor(text= text, return_tensors="pt")
speaker_embed = speaker_embeddings

In [ ]:
speech = model.generate(
    inputs["input_ids"],
    speaker_embeddings=speaker_embed,
    vocoder=vocoder
)

Display audio

In [60]:
speech_numpy = speech.cpu().numpy()
Audio(speech_numpy, rate=15000)
